In [1]:
from keras import Sequential
from keras.callbacks import Callback
from keras.layers import Embedding, np
from keras.utils import to_categorical
from keras_applications.densenet import layers
from keras_preprocessing import sequence
from keras_preprocessing.text import Tokenizer
from keras import backend as K

from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import jieba

Using TensorFlow backend.
/home/alexliu-ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 导入数据

In [2]:
data = pd.read_csv('../data/trainingset/sentiment_analysis_trainingset.csv')

### 数据探索

In [3]:
list(data.columns)[2:]

['location_traffic_convenience',
 'location_distance_from_business_district',
 'location_easy_to_find',
 'service_wait_time',
 'service_waiters_attitude',
 'service_parking_convenience',
 'service_serving_speed',
 'price_level',
 'price_cost_effective',
 'price_discount',
 'environment_decoration',
 'environment_noise',
 'environment_space',
 'environment_cleaness',
 'dish_portion',
 'dish_taste',
 'dish_look',
 'dish_recommendation',
 'others_overall_experience',
 'others_willing_to_consume_again']

In [ ]:
data['content'][0]

In [ ]:
list(jieba.cut(data['content'][0]))[1:-1]

In [ ]:
data.shape

### 数据分析

#### 1、数据预处理

In [ ]:
# 准备数据集
train_data = pd.read_csv('../data/trainingset/sentiment_analysis_trainingset.csv')
val_data = pd.read_csv('../data/validationset/sentiment_analysis_validationset.csv')
testa_data = pd.read_csv('../data/testa/sentiment_analysis_testa.csv')

In [ ]:
def make_stoplist():
    # 准备停用词
    with open('./stopwords.txt') as f:
        stoplist = f.read().split()
    stoplist.append(str("\n"))
    stoplist.append(" ")
    return stoplist

# 去掉停用词
def remove_stopword(data, stoplist):
    texts = []
    for index, row in data.iterrows():
        line = [word for word in list(row['words']) if word not in stoplist]
        texts.append(line)
    return texts

In [ ]:
# 划分小额数据做训练以及验证， 这部分可以不再使用
train_counts = 500
val_counts = 100
test_counts = 200
data_ls = data.loc[:train_counts + val_counts + test_counts-1, :]
data_ls['words'] = data_ls['content'].apply(lambda x: list(jieba.cut(x)))
y_cols = list(data_ls.columns)[2:]
print(data_ls.shape)
train = data_ls.loc[:train_counts-1, :]
val = data_ls.loc[train_counts:train_counts+val_counts-1, :]
test = data_ls.loc[train_counts+val_counts:train_counts+val_counts+test_counts-1, :]

In [ ]:
# jieba分词
train_data['words'] = train_data['content'].apply(lambda x: list(jieba.cut(x)))
val_data['words'] = testa_data['content'].apply(lambda x: list(jieba.cut(x)))
testa_data['words'] = testa_data['content'].apply(lambda x: list(jieba.cut(x)))

In [ ]:
# 去掉content列
%time
train_data = train_data.drop(columns=['content'], axis=1)
val_data = val_data.drop(columns=['content'], axis=1)
testa = testa_data.drop(columns=['content'], axis=1)

In [ ]:
# 去除停词
stoplist = make_stoplist()
train_data['texts'] = pd.Series(remove_stopword(train_data, stoplist))
val_data['texts'] = pd.Series(remove_stopword(val_data, stoplist))
testa_data['texts'] = pd.Series(remove_stopword(testa_data, stoplist))

In [ ]:
# 去掉words列
%time
train_data = train_data.drop(columns=['words'], axis=1)
val_data = val_data.drop(columns=['words'], axis=1)
testa = testa_data.drop(columns=['words'], axis=1)

In [ ]:
# 将预处理的数据放入到相应的csv中
train_data.to_csv('../data/trainingset/dealed_trainingset.csv')
val_data.to_csv('../data/validationset/dealed_validationset.csv')
testa_data.to_csv('../data/testa/dealed_testa.csv')